## Primero generando los datos adversarios

In [41]:
import torch
import numpy as np
import pandas as pd
import random 

In [42]:
def loadModelPytorch(model_path):
    """
    Loads a model from a .pth file.
    """
    l1 = 10
    l2 = 24
    l3 = 72
    l4 = 20
    model = torch.nn.Sequential(
    torch.nn.Linear(l1, l2),
    torch.nn.ReLU(),
    torch.nn.Linear(l2, l3),
    torch.nn.ReLU(),
    torch.nn.Linear(l3,l4)
    )
    loss_fn = torch.nn.MSELoss()
    learning_rate = 1e-3

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    gamma = 0.9
    epsilon = 1.0
    
    
    # Load the model
    model.load_state_dict(torch.load(model_path))
    # Set the model to evaluation mode
    model.eval()
    return model

In [43]:
model = loadModelPytorch('DRLMalwareModificator.sav')

In [44]:
model.eval()

Sequential(
  (0): Linear(in_features=10, out_features=24, bias=True)
  (1): ReLU()
  (2): Linear(in_features=24, out_features=72, bias=True)
  (3): ReLU()
  (4): Linear(in_features=72, out_features=20, bias=True)
)

## cargando los datos maliciosos

In [45]:
data=pd.read_csv('datasets/malicious_data_generated.csv')
npdata=data.to_numpy()
malData=np.copy(npdata)
print(type(malData[1,:]))
print(malData[:1].shape)

<class 'numpy.ndarray'>
(1, 10)


In [53]:
state1 = torch.from_numpy(np.asarray(malData[0,:])).float()
res=model(state1)
res_=res.data.numpy()
action=np.argmax(res_)

In [54]:
action

18

In [51]:
state1

tensor([-60., -35., -35., -69.,  73.,  61., -62.,  82., -44.,  59.])

In [50]:
res

tensor([ 2.6923,  3.9729,  8.9161,  6.2518,  6.6089,  8.7326,  5.9222,  3.9773,
         6.7195,  4.4702,  4.9236,  4.3771,  1.5577,  5.5841,  3.8655,  4.1586,
         4.6408,  5.8572, 10.0070,  8.2767], grad_fn=<AddBackward0>)

In [55]:
for x in range(0,len(malData)):
    state = torch.from_numpy(np.asarray(malData[x,:])).float()
    res=model(state1)
    valid=False
    while valid :
        res_=res.data.numpy()
        action=np.argmax(res_)
        if action == 0:
            valid = True
        else:
            malData[x,action] = malData[x,action] + 0.1
            state1 = torch.from_numpy(np.asarray(malData[x,:])).float()
            res=model(state1)
            valid = False
    

NameError: name 'valid' is not defined